In [4]:
import pandas as pd

# Set display options to show more rows and columns
pd.set_option('display.max_columns', None) # Show all columns

listings = pd.read_csv('/Users/krys/Desktop/UNIVERSITY/Year 1/Data Mining and Machine Learning/AirBnb datasets/barcelona/listings.csv')
#reviews = pd.read_csv('/Users/krys/Desktop/UNIVERSITY/Year 1/Data Mining and Machine Learning/AirBnb-Price-Prediction/dataset/reviews.csv')

/Users/krys/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# Identify columns where all values are the same
constant_columns = [col for col in listings.columns if listings[col].nunique() == 1]

# Print the columns where all values are the same
print("Columns where all values are the same:", constant_columns)

In [ ]:
# Remove columns where all values are the same
columns_to_remove = ['scrape_id', 'last_scraped', 'calendar_last_scraped']
listings = listings.drop(columns=columns_to_remove)

In [ ]:
# Remove irrelevant or uninformative features
columns_to_remove = ['id', 'listing_url', 'source', 'name', 'description', 'neighborhood_overview',
                     'picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                     'host_picture_url', 'license', 'calendar_updated']

listings = listings.drop(columns=columns_to_remove)

In [ ]:
# Convert boolean values into binary representations
listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].replace({'t': 1, 'f': 0})
listings['host_identity_verified'] = listings['host_identity_verified'].replace({'t': 1, 'f': 0})
listings['has_availability'] = listings['has_availability'].replace({'t': 1, 'f': 0})

In [ ]:
# Remove the dollar symbol and commas, then convert the column to numeric
listings['price'] = listings['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float)


Importing amenities from "amenities.csv" dataset

In [5]:
amenities_dummies = pd.read_csv('amenities.csv')

amenities_dummies

,Avena,Bar,Bidet,Bikes,Blender,Boat_slip,Bowling_alley,Bread_maker,Canal_view,Candy_-_FCS100X/E_-_Horno_el\u00e9ctrico_-_2100W,Climbing_wall,Desert_view,Dishes_and_silverware,Foosball_table,Gated_property,Golf_course_view,Hammock,Hockey_rink,Ikea,Ironing_board,Kayak,Kenwood,Lake_view,Laser_tag,Lavandin,Lockbox,Marca_blanca,Marina_view,Media_room,Mini_golf,Mosquito_net,Mountain_view,Nevera_y_congelador,Noise_decibel_monitors_on_property,Palmolive,Pantene,Piano,Private_sauna,Property_manager,Record_player,Rice_maker,River_view,Samsung,Sauna,Sea_view,Sempre_hi_ha_xamp\u00fa,Shared_sauna,Siemens_Inoxidable_con_Vapor,Skate_ramp,Ski-in/Ski-out,Sonos_(Spotify_Connect,Sony,Sunstech,Table_corner_guards,Theme_room,Toaster,Valley_view,Vineyard_view,Waterfront,1_day_a_week_-_available_at_extra_cost,2_days_a_week_-_available_at_extra_cost,2-5_years_old,4_days_a_week_-_available_at_extra_cost,5_days_a_week_-_available_at_extra_cost,5-10_years_old,AirPlay,Apple,Fructis,Inox,Nivea,PS2,Puerta_desmontable_de_doble_cristal_con_efecto_espejo,Thursday_-_available_at_extra_cost,Tuesday,Wednesday,Unnamed: 75,and_10+_years_old,and_5-10_years_old,and_PS3,and_dresser,and_wardrobe,closet,every_day_-_available_at_extra_cost,every_day_-_included_with_your_stay,free_weights,olympic-sized,peque\u00f1a,rowing,wardrobe,water_slide,wood-burning,wifi_aggregated,shampoo_aggregated,body_soap_aggregated,tv_aggregated,refrigerator_aggregated,oven_aggregated,stove_aggregated,sound_system_aggregated,bbq_aggregated,games_aggregated,baby_aggregated,baking_aggregated,booster_seat_aggregated,ceiling_fan_aggregated,changing_table_aggregated,city_skyline_view_aggregated,clothing_storage_aggregated,coffee_aggregated,crib_aggregated,dryer_aggregated,charger_aggregated,dove_aggregated,elevator_aggregated,essentials_aggregated,ethernet_aggregated,exercise_equipment_aggregated,cameras_aggregated,fire_extinguisher_aggregated,first_kit_aggregated,fire_pit_aggregated,high_chair_aggregated,carport_aggregated,parking_aggregated,washer_aggregated,freezer_aggregated,garden_aggregated,housekeeping_aggregated,keypad_aggregated,kitchen_aggregated,lock_on_bedroom_aggregated,cleaning_aggregated,pool_aggregated,long_term_aggregated,luggage_dropoff_aggregated,microwave_aggregated,outdoor_aggregated,ocean_view_aggregated,gym_aggregated,hot_tub_aggregated,bay_aggregated,garage_aggregated,resort_aggregated,hangers_aggregated,heating_aggregated,host_greets_aggregated,hot_water_aggregated,pets_aggregated,private_entrance_aggregated,balcony_aggregated,safe_aggregated,self_check_in_aggregated,single_level_home_aggregated,smoke_alarm_aggregated,smoking_allowed_aggregated,sun_loungers_aggregated,prime_video_aggregated,chromecast_aggregated,dvd_aggregated,monday_to_friday_aggregated,disney_aggregated,netflix_aggregated,roku_aggregated,saturday_aggregated,sunday_aggregated,bathtub_aggregated,bed_linens_aggregated,books_and_reading_material_aggregated,breakfast_aggregated,building_staff_aggregated,carbon_monoxide_alarm_aggregated,cooking_basics_aggregated,courtyard_view_aggregated,dining_table_aggregated,extra_pillows_and_blankets_aggregated,harbor_view_aggregated,iron_aggregated,laundromat_nearby_aggregated,mini_fridge_aggregated,movie_theater_aggregated,outlet_covers_aggregated,park_view_aggregated,ping_pong_table_aggregated,portable_fans_aggregated,portable_heater_aggregated,private_living_room_aggregated,room-darkening_shades_aggregated,shower_gel_aggregated,smart_lock_aggregated,window_guards_aggregated,wine_glasses_aggregated,french_press_aggregated,heated_aggregated,infinity_aggregated,open_24_hours_aggregated,open_specific_hours_aggregated,premium_cable_aggregated,rooftop_aggregated,saltwater_aggregated,standard_cable_aggregated,stationary_bike_aggregated,treadmill_aggregated,workout_bench_aggregated,yoga_mat_aggregated
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [6]:
# Concatenate the original DataFrame (excluding the old 'amenities' column) with the new boolean features
listings = pd.concat([listings.drop(columns=['amenities']), amenities_dummies], axis=1)

listings

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,Avena,Bar,Bidet,Bikes,Blender,Boat_slip,Bowling_alley,Bread_maker,Canal_view,Candy_-_FCS100X/E_-_Horno_el\u00e9ctrico_-_2100W,Climbing_wall,Desert_view,Dishes_and_silverware,Foosball_table,Gated_property,Golf_course_view,Hammock,Hockey_rink,Ikea,Ironing_board,Kayak,Kenwood,Lake_view,Laser_tag,Lavandin,Lockbox,Marca_blanca,Marina_view,Media_room,Mini_golf,Mosquito_net,Mountain_view,Nevera_y_congelador,Noise_decibel_monitors_on_property,Palmolive,Pantene,Piano,Private_sauna,Property_manager,Record_player,Rice_maker,River_view,Samsung,Sauna,Sea_view,Sempre_hi_ha_xamp\u00fa,Shared_sauna,Siemens_Inoxidable_con_Vapor,Skate_ramp,Ski-in/Ski-out,Sonos_(Spotify_Connect,Sony,Sunstech,Table_corner_guards,Theme_room,Toaster,Valley_view,Vineyard_view,Waterfront,1_day_a_week_-_available_at_extra_cost,2_days_a_week_-_available_at_extra_cost,2-5_years_old,4_days_a_week_-_available_at_extra_cost,5_days_a_week_-_available_at_extra_cost,5-10_years_old,AirPlay,Apple,Fructis,Inox,Nivea,PS2,Puerta_desmontable_de_doble_cristal_con_efecto_espejo,Thursday_-_available_at_extra_cost,Tuesday,Wednesday,Unnamed: 75,and_10+_years_old,and_5-10_years_old,and_PS3,and_dresser,and_wardrobe,closet,every_day_-_available_at_extra_cost,every_day_-_included_with_your_stay,free_weights,olympic-sized,peque\u00f1a,rowing,wardrobe,water_slide,wood-burning,wifi_aggregated,shampoo_aggregated,body_soap_aggregated,tv_aggregated,refrigerator_aggregated,oven_aggregated,stove_aggregated,sound_system_aggregated,bbq_aggregated,games_aggregated,baby_aggregated,baking_aggregated,booster_seat_aggregated,ceiling_fan_aggregated,changing_table_aggregated,city_skyline_view_aggregated,clothing_storage_aggregated,coffee_aggregated,crib_aggregated,dryer_aggregated,charger_aggregated,dove_aggregated,elevator_aggregated,essentials_aggregated,ethernet_aggregated,exercise_equipment_aggregated,cameras_aggregated,fire_extinguisher_aggregated,first_kit_aggregated,fire_pit_aggregated,high_chair_aggregated,carport_aggregated,parking_aggregated,washer_aggregated,freezer_aggregated,garden_aggregated,housekeeping_aggregated,keypad_aggregated,kitchen_aggregated,lock_on_bedroom_aggregated,cleaning_aggregated,pool_aggregated,long_term_aggregated,luggage_dropoff_aggregated,microwave_aggregated,outdoor_aggregated,ocean_view_aggregated,gym_aggregated,hot_tub_aggregated,bay_aggregated,garage_aggregated,resort_aggregated,hangers_aggregated,heating_aggregated,host_greets_aggregated,hot_water_aggregated,pets_aggregated,private_entrance_aggregated,balcony_aggregated,safe_aggregated,self_check_in_aggregated,single_level_home_aggregated,smoke_alarm_aggregated,smoking_allowed_aggregated,sun_loungers_aggregated,prime_video_aggr

In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming `listings` is your DataFrame

# Select numeric columns for scaling
numeric_columns = listings.select_dtypes(include=['float64', 'int64']).columns
numeric_data = listings[numeric_columns]

# Scale the numeric data
scaler = StandardScaler()
numeric_data_scaled = scaler.fit_transform(numeric_data)

# Convert the scaled data back to a DataFrame
numeric_data_scaled_df = pd.DataFrame(numeric_data_scaled, columns=numeric_columns)

# Combine scaled numeric data with non-numeric data
non_numeric_data = listings.drop(columns=numeric_columns)
listings = pd.concat([numeric_data_scaled_df, non_numeric_data.reset_index(drop=True)], axis=1)

listings